<a href="https://colab.research.google.com/github/Minakshi654/Modelname/blob/main/Text%20Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset('daily_dialog')

In [ ]:
dataset

In [ ]:
# Combining Text
def conv_to_list_function(examples):
    return ["".join(x) for x in examples]

#cleaning first 1000 dialogs
text_list = conv_to_list_function(dataset['train'][:1000]['dialog'])

In [ ]:
len(text_list)

In [ ]:
text = ''
for x in text_list:
  text += x
print('Length of the full text: ', len(text))
print('Printing first 150 characters of the string: \n', text[:150])

In [ ]:
with open('input.txt', 'w') as file:
  file.write(text)

In [ ]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling, TFAutoModelForCausalLM
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

In [ ]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["dialog"]])

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns = dataset["train"].column_names,
)

In [ ]:
tokenized_dataset

In [ ]:
block_size = 128

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)
lm_dataset

In [ ]:

from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")


In [ ]:
from transformers import TFAutoModelForCausalLM
model = TFAutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

tf_train_set = model.prepare_tf_dataset(
    lm_dataset["train"],
    shuffle = True,
    batch_size = 32,
    collate_fn = data_collator,
)

tf_val_set = model.prepare_tf_dataset(
    lm_dataset["validation"],
    shuffle = False,
    batch_size = 32,
    collate_fn = data_collator,
)

In [ ]:
tf_train_set

In [ ]:
tf_val_set

In [ ]:
from transformers import TFAutoModelForCausalLM, create_optimizer
from tensorflow.keras.optimizers import Adam # Import Adam from tf.keras.optimizers

# Create a Hugging Face compatible optimizer using create_optimizer
num_train_steps = len(tf_train_set) # Replace with the actual number of training steps
optimizer, lr_schedule = create_optimizer(
    init_lr=0.001,
    num_warmup_steps=0,
    num_train_steps=num_train_steps
)

# Compile the model using the Hugging Face compatible optimizer
model.compile(optimizer=optimizer)
model.summary()

In [ ]:
history = model.fit(x=tf_train_set, validation_data=tf_val_set, epochs=3)

Epoch 1/3
 29/348 [=>............................] - ETA: 6:47:12 - loss: 4.1978

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
input = 'I feel great today. It must be a warm and sunny day'
input_ids = tokenizer.encode(input, return_tensors="tf")
input_ids

In [ ]:
output_ids = model.generate(input_ids, max_length=100, num_beams=5, no_repeat_ngram_size=2)
output_ids

In [ ]:
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
output

In [ ]:
input_ids = tokenized_dataset['test'][0]['input_ids']
input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
input_text

In [ ]:
input_ids = tokenizer.encode(input_text, return_tensors='tf')
input_ids

In [ ]:
output_ids =  model.generate(input_ids, max_length=input_ids.shape[1]+50, num_beams=5, no_repeat_ngram_size=2)
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
output

In [ ]:
logits = model(input_ids).logits
probs = tf.nn.softmax(logits, axis=-1)
target_ids = input_ids[:, 1:]
loss = tf.keras.losses.sparse_categorical_crossentropy(target_ids, logits[:, :-1, :])
perplexity = tf.exp(tf.reduce_mean(loss))

print("Output:", output)
print("Perplexity:", perplexity.numpy())

In [ ]:
input_ids = tokenized_dataset['test'][:10]['input_ids']

perplexities = []
outputs = []

for input_id in input_ids:
  input_text = tokenizer.decode(input_id, skip_special_tokens=True)
  temp_input_id = tokenizer.encode(input_text, return_tensors='tf')
  output_ids =  model.generate(temp_input_id, max_length=temp_input_id.shape[1]+50, num_beams=5, no_repeat_ngram_size=2)
  output = tokenizer.decode(output_ids[0][-50:], skip_special_tokens=True)
  outputs.append({
      'input' : input_text,
      'output' : output
  })

  #Evaluation
  logits = model(temp_input_id).logits
  probs = tf.nn.softmax(logits, axis=-1)
  target_id = temp_input_id[:, 1:]
  loss = tf.keras.losses.sparse_categorical_crossentropy(target_id, logits[:, :-1, :])
  perplexity = tf.exp(tf.reduce_mean(loss))
  perplexities.append(perplexity.numpy())
  print('done')

In [ ]:
# an output sample
outputs[6]

In [ ]:
perplexities = np.array(perplexities)
np.mean(perplexities)